In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing
import csv
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

pd.options.mode.chained_assignment = None
data = pd.read_csv('StatsAfterAllNBAInsertion.csv')

In [3]:
data.describe()

,Year,Age,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,MPpG,PER,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
count,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15095.000000,...,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000
mean,2004.197550,26.984834,0.027483,0.276358,0.430993,48.339603,23.073907,1130.345695,19.823603,12.399821,...,36.770662,0.642278,23.103245,0.398854,67.158808,1.190523,101.926821,1.864801,461.931854,7.824821
std,7.998757,4.149146,0.163493,1.305299,1.765006,26.478508,28.236547,915.995235,10.293082,6.282646,...,36.014443,0.466536,34.770722,0.497782,62.887391,0.794410,76.944305,0.851711,461.987449,5.834517
min,1990.000000,18.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-90.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1997.000000,24.000000,0.000000,0.000000,0.000000,25.000000,0.000000,292.000000,11.200000,9.600000,...,8.000000,0.300000,3.000000,0.100000,16.000000,0.600000,32.000000,1.200000,86.000000,3.300000
50%,2005.000000,26.000000,0.000000,0.000000,0.000000,52.000000,8.000000,940.000000,18.900000,12.600000,...,27.000000,0.600000,11.000000,0.200000,50.000000,1.000000,92.000000,1.900000,317.000000,6.300000
75%,2011.000000,30.000000,0.000000,0.000000,0.000000,74.000000,43.000000,1835.250000,28.300000,15.600000,...,55.000000,0.900000,28.000000,0.500000,102.000000,1.600000,159.000000,2.500000,713.250000,11.200000
max,2017.000000,44.000000,1.000000,15.000000,15.000000,85.000000,83.000000,3533.000000,44.500000,129.100000,...,246.000000,3.500000,376.000000,6.000000,464.000000,5.700000,371.000000,6.000000,2832.000000,35.400000


In [4]:
# Fill in blanks/NAs

data['PER'] = data['PER'].fillna(data['PER'].median())
data['TS%'] = data['TS%'].fillna(data['TS%'].median())
data['3PAr'] = data['3PAr'].fillna(data['3PAr'].median())
data['FTr'] = data['FTr'].fillna(data['FTr'].median())
data['ORB%'] = data['ORB%'].fillna(data['ORB%'].median())
data['DRB%'] = data['DRB%'].fillna(data['DRB%'].median())
data['TRB%'] = data['TRB%'].fillna(data['TRB%'].median())
data['AST%'] = data['AST%'].fillna(data['AST%'].median())
data['STL%'] = data['STL%'].fillna(data['STL%'].median())
data['BLK%'] = data['BLK%'].fillna(data['BLK%'].median())
data['TOV%'] = data['TOV%'].fillna(data['TOV%'].median())
data['USG%'] = data['USG%'].fillna(data['USG%'].median())
data['WS/48'] = data['WS/48'].fillna(data['WS/48'].median())
data['FG%'] = data['FG%'].fillna(data['FG%'].median())
data['3P%'] = data['3P%'].fillna(data['3P%'].median())
data['2P%'] = data['2P%'].fillna(data['2P%'].median())
data['eFG%'] = data['eFG%'].fillna(data['eFG%'].median())
data['FT%'] = data['FT%'].fillna(data['FT%'].median())


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15100 entries, 0 to 15099
Data columns (total 68 columns):
Year                    15100 non-null int64
Player                  15100 non-null object
Age                     15100 non-null int64
Pos                     15100 non-null object
All NBA                 15100 non-null int64
All NBA Acc             15100 non-null int64
All NBA Total Career    15100 non-null int64
G                       15100 non-null int64
GS                      15100 non-null int64
MP                      15100 non-null int64
MPpG                    15100 non-null float64
PER                     15100 non-null float64
TS%                     15100 non-null float64
3PAr                    15100 non-null float64
FTr                     15100 non-null float64
ORB%                    15100 non-null float64
DRB%                    15100 non-null float64
TRB%                    15100 non-null float64
AST%                    15100 non-null float64
STL%            

In [5]:
data.Pos.unique()

array(['PF', 'SF', 'C', 'SG', 'PG', 'SG-SF', 'PF-C', 'SF-PF', 'PF-SF',
       'SG-PG', 'PG-SG', 'C-PF', 'C-SF', 'SF-SG', 'PG-SF', 'SG-PF'],
      dtype=object)

In [6]:
#Give Position numerical labels

le = preprocessing.LabelEncoder()
data['Pos'] = le.fit_transform(data['Pos'].astype(str))
le_Pos_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_Pos_mapping)

{'C': 0, 'C-PF': 1, 'C-SF': 2, 'PF': 3, 'PF-C': 4, 'PF-SF': 5, 'PG': 6, 'PG-SF': 7, 'PG-SG': 8, 'SF': 9, 'SF-PF': 10, 'SF-SG': 11, 'SG': 12, 'SG-PF': 13, 'SG-PG': 14, 'SG-SF': 15}


In [7]:
# Delete duplicate Data that occured when Player transferred mid season

check = 0
index = 0
for year in data['Year']:
    if (check != year):
        check = year
        index = index + 1
        continue
    #remove row using index
    data = data.drop(index)
    index = index + 1

In [8]:
# Check if duplicate deletion was successful

def checkForDuplicates(dataframe):
    check = 0
    count = 0
    for year in dataframe['Year']:
        if (count == 2):
            return True
        if (check != year):
            check = year
            count = 0
            continue
        count = count + 1
        continue
    return False

In [9]:
print(checkForDuplicates(data))

False


In [10]:
# Split Dataset into sub-Datasets containing information about single players and careers per Dataframe

subDataSets = []
count = 0
string = 'A.C. Green'
for i,k in data.iterrows():
    if (string != k['Player']):
        subDataSets.append(data.loc[count:i-1])
        count = i
        string = k['Player']
    if (i == len(data)-1):
        subDataSets.append(data.loc[count:i])

In [11]:
# NOT Optional, only use players with at least 5 entries

careerPlayers = []
for subset in subDataSets:
    if (len(subset) >= 5):
        careerPlayers.append(subset)

In [12]:
# Loop through every single player's career and take averages of all features
# Use mode to determine the most played position
# Write result into a new csv file for further usage

with open('AverageStats.csv', 'w') as f1:
    for dataFrame in careerPlayers:
        dfTmp = dataFrame.groupby('Player').mean()
        dfTmp['Pos'] = dataFrame.mode()['Pos'][0]
        # Initially left out
        # dfTmp['Age'] = dataFrame.max()['Age']
        dfTmp.round(2).to_csv(f1, header=False)

In [35]:
# Open Average Stats over entire career include names of columns/headers

averageData = pd.read_csv('AverageStats.csv',names=['Player','Year','Age','Pos','All NBA','All NBA Acc','All NBA Total Career','G','GS','MP','MPpG','PER','TS%',
       '3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','OWS','DWS','WS','WS/48','OBPM','DBPM','BPM',
       'VORP','FG','FGpG','FGA','FGApG','FG%','3P','3PpG','3PA','3PApG','3P%','2P','2PA','2P%','eFG%','FT','FTpG','FTA',
       'FTApG','FT%','ORB','ORBpG','DRB','DRBpG','TRB','TRBpG','AST','ASTpG','STL','STLpG','BLK','BLKpG','TOV','TOVpG',
       'PF','PFpG','PTS','PTSpG'])
averageData.head()

,Player,Year,Age,Pos,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,A.C. Green,1995.50,31.50,3.0,0.0,0.0,0.0,79.42,51.08,2302.00,...,61.08,0.77,26.42,0.33,82.83,1.02,138.33,1.72,744.42,9.23
1,A.J. Price,2012.50,25.50,6.0,0.0,0.0,0.0,43.50,4.17,654.83,...,20.83,0.43,1.50,0.03,38.17,0.80,39.50,0.82,253.83,5.35
2,Aaron Brooks,2012.56,27.56,6.0,0.0,0.0,0.0,68.11,20.22,1472.22,...,39.22,0.57,9.67,0.14,109.89,1.56,132.11,1.89,687.11,9.61
3,Aaron Gray,2011.00,26.00,0.0,0.0,0.0,0.0,45.43,12.43,550.14,...,13.43,0.30,12.43,0.26,37.86,0.81,96.86,2.09,152.29,3.24
4,Aaron McKie,2001.00,28.00,12.0,0.0,0.0,0.0,61.00,22.08,1473.54,...,70.15,1.03,12.62,0.20,81.46,1.22,122.15,1.85,451.62,6.63


In [36]:
# Drop irrelevant features

averageData = averageData.drop('Player', axis=1)
averageData = averageData.drop('Year', axis=1)
# maybe drop age maybe not 
averageData = averageData.drop('Age', axis=1)
averageData = averageData.drop('All NBA', axis=1)
averageData = averageData.drop('All NBA Acc', axis=1)
averageData.head()

,Pos,All NBA Total Career,G,GS,MP,MPpG,PER,TS%,3PAr,FTr,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,3.0,0.0,79.42,51.08,2302.00,28.65,13.81,0.53,0.06,0.42,...,61.08,0.77,26.42,0.33,82.83,1.02,138.33,1.72,744.42,9.23
1,6.0,0.0,43.50,4.17,654.83,13.77,11.72,0.48,0.47,0.17,...,20.83,0.43,1.50,0.03,38.17,0.80,39.50,0.82,253.83,5.35
2,6.0,0.0,68.11,20.22,1472.22,20.84,12.79,0.52,0.41,0.19,...,39.22,0.57,9.67,0.14,109.89,1.56,132.11,1.89,687.11,9.61
3,0.0,0.0,45.43,12.43,550.14,11.99,11.56,0.53,0.00,0.38,...,13.43,0.30,12.43,0.26,37.86,0.81,96.86,2.09,152.29,3.24
4,12.0,0.0,61.00,22.08,1473.54,22.26,11.73,0.50,0.20,0.21,...,70.15,1.03,12.62,0.20,81.46,1.22,122.15,1.85,451.62,6.63


In [37]:
# Create correlation matrix

corr_matrix = averageData.corr().abs()

# Select upper triangle of correlation matrix

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95

to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

['MPpG',
 'TRB%',
 'WS',
 'FGpG',
 'FGA',
 'FGApG',
 '3PpG',
 '3PA',
 '3PApG',
 '2P',
 '2PA',
 'FTpG',
 'FTA',
 'FTApG',
 'ORBpG',
 'DRBpG',
 'TRB',
 'TRBpG',
 'ASTpG',
 'STLpG',
 'BLKpG',
 'TOVpG',
 'PTS',
 'PTSpG']

In [38]:
# Drop the highly corelated features from dataframe

for feature in to_drop:
    if (feature == 'All NBA Total Career'):
        continue
    averageData = averageData.drop(feature, axis=1)

In [39]:
# Split dataframe into train and test sets aiming for ALL NBA TOTAL CAREER

X = averageData.drop("All NBA Total Career", axis=1)
y = averageData["All NBA Total Career"]

test_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=42)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(803, 38) (803,)
(201, 38) (201,)


In [40]:
# Train the linear regression algorithm using default parameters

r = LinearRegression()  
r.fit(X_train, y_train)
y_pred = r.predict(X_test)
df1 = pd.DataFrame({'Actual': y_test, 'Predicted All NBA': y_pred})
df1

,Actual,Predicted All NBA
926,0.0,-0.095252
630,0.0,1.243153
682,0.0,-0.484721
514,0.0,-0.154912
365,0.0,2.080030
655,0.0,-0.296726
656,0.0,0.956678
529,0.0,0.992840
321,0.0,-0.558625
70,0.0,-0.048937


In [19]:
r.coef_

array([ 2.77217462e-03, -1.74624937e-02, -1.32611052e-02,  1.17588780e-01,
       -1.15567900e+01,  1.74346687e+00, -1.81667037e-01, -1.44668668e-02,
       -3.10770623e-02, -3.34448234e-01, -2.42818883e-01,  7.88934661e-02,
        8.49677270e-02,  8.79539002e-01,  7.44941487e-01,  9.53135295e-02,
        2.21978697e+00, -1.24161584e-02,  1.17730526e-01, -2.81578748e+00,
       -1.15243566e-02, -2.55779204e-04,  1.10970983e-02, -2.02154876e-03])

In [41]:
r2_score(y_test, y_pred) 

0.6431526859518133

In [42]:
mean_squared_error(y_test, y_pred)

0.7472056478607063

In [43]:
dataCurrent = pd.read_csv('2019Data.csv')
dataCurrent.head()

,Year,Player,Age,Pos,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,2019,Luka Doncic,20,9,0,0,0,72,72,2318,...,77,1.1,25,0.3,247,3.4,137,1.9,1526,21.2
1,2016,Karl-Anthony Towns,20,0,0,0,2,82,82,2627,...,58,0.7,138,1.7,183,2.2,245,3.0,1503,18.3
2,2017,Karl-Anthony Towns,21,0,0,0,2,82,82,3030,...,57,0.7,103,1.3,212,2.6,241,2.9,2061,25.1
3,2018,Karl-Anthony Towns,22,0,1,1,2,82,82,2918,...,64,0.8,115,1.4,159,1.9,285,3.5,1743,21.3
4,2019,Karl-Anthony Towns,23,0,1,2,2,77,77,2545,...,67,0.9,125,1.6,240,3.1,292,3.8,1880,24.4


In [23]:
# Create new csv file with Towns', Irving's, Doncic's, and Curry's current career average stats  

with open('AverageStats2019.csv', 'w') as f2:
    dfTmp = dataCurrent.groupby('Player').mean()
    dfTmp.round(2).to_csv(f2, header=False)

In [44]:
# Open Average Stats 2019 over entire career include names of columns/headers

Average2019 = pd.read_csv('AverageStats2019.csv',names=['Player','Year','Age','Pos','All NBA','All NBA Acc','All NBA Total Career','G','GS','MP','MPpG','PER','TS%',
       '3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','OWS','DWS','WS','WS/48','OBPM','DBPM','BPM',
       'VORP','FG','FGpG','FGA','FGApG','FG%','3P','3PpG','3PA','3PApG','3P%','2P','2PA','2P%','eFG%','FT','FTpG','FTA',
       'FTApG','FT%','ORB','ORBpG','DRB','DRBpG','TRB','TRBpG','AST','ASTpG','STL','STLpG','BLK','BLKpG','TOV','TOVpG',
       'PF','PFpG','PTS','PTSpG'])
Average2019.head()

,Player,Year,Age,Pos,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,Karl-Anthony Towns,2017.5,21.50,0.0,0.50,0.75,2.0,80.75,80.75,2780.00,...,61.5,0.78,120.25,1.50,198.50,2.45,265.75,3.30,1796.75,22.28
1,Kyrie Irving,2015.5,22.75,6.0,0.25,0.75,2.0,63.50,63.50,2146.12,...,84.0,1.31,22.25,0.35,167.75,2.65,139.75,2.20,1411.75,16.60
2,Luka Doncic,2019.0,20.00,9.0,0.00,0.00,0.0,72.00,72.00,2318.00,...,77.0,1.10,25.00,0.30,247.00,3.40,137.00,1.90,1526.00,21.20
3,Stephen Curry,2014.5,25.50,6.0,0.60,2.10,6.0,69.40,68.80,2385.90,...,120.1,1.69,15.40,0.24,216.30,3.07,172.10,2.46,1631.50,23.06


In [45]:
# Drop irrelevant features

Average2019 = Average2019.drop('Player', axis=1)
Average2019 = Average2019.drop('Year', axis=1)
Average2019 = Average2019.drop('Age', axis=1)
Average2019 = Average2019.drop('All NBA', axis=1)
Average2019 = Average2019.drop('All NBA Acc', axis=1)
Average2019.head()

,Pos,All NBA Total Career,G,GS,MP,MPpG,PER,TS%,3PAr,FTr,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,0.0,2.0,80.75,80.75,2780.00,34.42,24.92,0.62,0.20,0.30,...,61.5,0.78,120.25,1.50,198.50,2.45,265.75,3.30,1796.75,22.28
1,6.0,2.0,63.50,63.50,2146.12,33.58,22.08,0.57,0.30,0.25,...,84.0,1.31,22.25,0.35,167.75,2.65,139.75,2.20,1411.75,16.60
2,9.0,0.0,72.00,72.00,2318.00,32.20,19.68,0.55,0.43,0.41,...,77.0,1.10,25.00,0.30,247.00,3.40,137.00,1.90,1526.00,21.20
3,6.0,6.0,69.40,68.80,2385.90,33.88,23.91,0.62,0.47,0.23,...,120.1,1.69,15.40,0.24,216.30,3.07,172.10,2.46,1631.50,23.06


In [46]:
# Drop previously highly correlated features from new dataframe

for feature in to_drop:
    if (feature == 'All NBA Total Career'):
        continue
    Average2019 = Average2019.drop(feature, axis=1)

In [47]:
X2019 = Average2019.drop('All NBA Total Career', axis=1)
y2019 = Average2019['All NBA Total Career']

In [48]:
# Use previous model to predict All NBA Total Career based on current career's stats

names = ['Towns', 'Irving', 'Doncic', 'Curry']
y2019_pred = r.predict(X2019)
df3 = pd.DataFrame({'Actual': names, 'Predicted All NBA': y2019_pred, 'Remaining': y2019_pred - y2019})
df3

,Actual,Predicted All NBA,Remaining
0,Towns,6.668295,4.668295
1,Irving,3.689998,1.689998
2,Doncic,4.215344,4.215344
3,Curry,5.705669,-0.294331


In [32]:
# Therefore, All NBA Selections remaining are calculated by subtracting the current accumulated All NBA Selections
# from the predicted Total Career rounded to the higher integer.


# Included every single player:
# All NBA included:

# Towns:     7 - 2 = 5
# Irving:    4 - 2 = 2
# Doncic:    0 - 0 = 0
# Curry:     8 - 6 = 2

# If All NBA is deleted:

# Towns:     6 - 2 = 4
# Irving:    4 - 2 = 2
# Doncic:    5 - 0 = 5
# Curry:     5 - 6 = -1 -> No remaining All NBA

# Included only 5 entry players:

# If All NBA is included:

# Towns:     7 - 2 = 5
# Irving:    3 - 2 = 1
# Doncic:    0 - 0 = 0
# Curry:     8 - 6 = 2

# If All NBA is dropped:

# Towns:     6 - 2 = 4
# Irving:    3 - 2 = 1
# Doncic:    4 - 0 = 4
# Curry:     5 - 6 = -1 -> No remaining All NBA